# Part 1: Initial Planning

## Robert, Dave and Stan
--------------------------------------------------------------------------------------------------------------------------------


### We hope to predict the winner of the 2018 NBA Playoffs 
### Collected data from 
  1. https://www.cbssports.com/nba/standings 
  2. https://www.kaggle.com/pablote/nba-enhanced-stats 
  3. https://stats.nba.com/teams/traditional/?sort=W_PCT&dir=-1&Season=2017-18&SeasonType=Regular%20Season 
                       

Import necessary libraries 

In [2]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

Read box statistics from 2016/2017 season and 2017/2018

In [3]:
team_box1617 = pd.read_csv("./2016-17_teamBoxScore.csv", sep=',', dtype = str)
player_box1617 = pd.read_csv("./2016-17_playerBoxScore.csv", sep=',', dtype = str)
official_box1617 = pd.read_csv("./2016-17_officialBoxScore.csv", sep=',', dtype = str)
standings1617 = pd.read_csv("./2016-17_standings.csv", sep=',', dtype = str)
team_box1718 = pd.read_csv("./2017-18_teamBoxScore.csv", sep=',', dtype = str)
player_box1718 = pd.read_csv("./2017-18_playerBoxScore.csv", sep=',', dtype = str)
official_box1718 = pd.read_csv("./2017-18_officialBoxScore.csv", sep=',', dtype = str)
standings1718 = pd.read_csv("./2017-18_standings.csv", sep=',', dtype = str)

Read csv file that has been scraped from cite 1.

In [4]:
names = ["Team", "W", "L", 'Pct', 'GB', 'Home', 'Road', "Conf", "Div", "Streak", "L10", "?"]
team_statistics = pd.read_csv("./scraping_attempt2.csv", sep = ',', dtype = str, names = names)
del team_statistics['?']
team_statistics.drop(0, inplace=True)
team_statistics.drop(16, inplace=True)
#team_statistics

Read csv files from end of year statistsics from the past 4 years

In [5]:
team_stats1415 = pd.read_csv("./Stats_14-15.csv", sep = ",")
team_stats1516 = pd.read_csv("./Stats_15-16.csv", sep = ",")
team_stats1617 = pd.read_csv("./Stats_16-17.csv", sep = ",")
team_stats1718 = pd.read_csv("./Stats_17-18.csv", sep = ",")

In [10]:
team_stats1415['GP'] = team_stats1415['W'] + team_stats1415['L']
team_stats1516['GP'] = team_stats1516['W'] + team_stats1516['L']
team_stats1617['GP'] = team_stats1617['W'] + team_stats1617['L']
team_stats1718['GP'] = team_stats1718['W'] + team_stats1718['L']

In [11]:
team_stats1516

,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PLUS/MINUS
0,Golden_State_Warriors,82,73,9,0.890,NaN,114.9,42.5,87.3,48.7,...,36.2,46.2,28.9,15.2,8.4,6.1,4.1,20.7,19.8,10.8
1,San_Antonio_Spurs,82,67,15,0.817,NaN,103.5,40.1,82.9,48.4,...,34.5,43.9,24.5,13.1,8.3,5.9,3.9,17.5,19.5,10.6
2,Cleveland_Cavaliers,82,57,25,0.695,NaN,104.3,38.7,84.0,46.0,...,33.9,44.5,22.7,13.6,6.7,3.9,4.4,20.3,20.6,6.0
3,Toronto_Raptors,82,56,26,0.683,NaN,102.7,36.7,81.3,45.1,...,33.2,43.4,18.7,13.1,7.8,5.5,5.4,19.6,22.0,4.5
4,Oklahoma_City_Thunder,82,55,27,0.671,NaN,110.2,41.1,86.4,47.6,...,35.6,48.6,23.0,15.9,7.4,5.9,4.5,20.6,20.2,7.3
5,LA_Clippers,82,53,29,0.646,NaN,104.5,38.3,82.4,46.5,...,33.3,42.0,22.8,13.0,8.6,5.6,3.2,21.3,22.5,4.3
6,Miami_Heat,82,48,34,0.585,NaN,100.0,38.4,81.7,47.0,...,34.3,44.1,20.8,14.1,6.7,6.5,4.1,18.3,19.6,1.6
7,Boston_Celtics,82,48,34,0.585,NaN,105.7,39.2,89.2,43.9,...,33.3,44.9,24.2,13.7,9.2,4.2,5.5,21.9,21.0,3.2
8,Charlotte_Hornets,82,48,34,0.585,NaN,103.4,37.0,84.4,43.9,...,35.0,43.9,21.7,12.5,7.3,5.3,5.5,18.1,20.4,2.7
9,Atlanta_Hawks,82,48,34,0.585,NaN,102.8,38.6,84.4,45.8,...,33.8,42.1,25.6,15.0,9.1,5.9,5.0,19.1,18.3,3.6
